<a href="https://colab.research.google.com/github/chauhan231405/bigdataLAB/blob/main/Spark4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=b0be4ee735a01b150b386d0f11d025247662932dc9bdcdc2b906c696c7e5279e
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
#Setup Spark Session
spark = SparkSession.builder.appName('Recommender').getOrCreate()
spark

In [ ]:
data = spark.read.csv('/content/book_ratings.csv',inferSchema=True,header=True)
data.show(5)

+-------+-------+------+
|book_id|user_id|rating|
+-------+-------+------+
|      1|    314|     5|
|      1|    439|     3|
|      1|    588|     5|
|      1|   1169|     4|
|      1|   1185|     4|
+-------+-------+------+
only showing top 5 rows



In [ ]:
data.count()

254812

In [ ]:
data.describe().show()

+-------+------------------+-----------------+------------------+
|summary|           book_id|          user_id|            rating|
+-------+------------------+-----------------+------------------+
|  count|            254812|           254812|            254812|
|   mean|1274.9756408646374|24688.24237477042| 3.825373216332041|
| stddev|  736.058995940083|14983.10160239284|1.0099376939054863|
|    min|                 1|                1|                 1|
|    max|              2550|            53424|                 5|
+-------+------------------+-----------------+------------------+



In [ ]:
train_data, test_data = data.randomSplit([0.8, 0.2])

In [ ]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5,regParam=0.01,userCol="user_id",itemCol="book_id",ratingCol="rating")
#Fitting the model on the train_data
model = als.fit(train_data)
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test_data)
#Displaying predictions calculated by the model
predictions.show()

+-------+-------+------+----------+
|book_id|user_id|rating|prediction|
+-------+-------+------+----------+
|      1|  32592|     4|  4.361717|
|      1|  48482|     3| 3.6524062|
|      1|  13544|     5| 4.0947537|
|      1|   3922|     5|  4.806741|
|      1|   6630|     5| 4.3079033|
|      1|  24845|     5| 4.8308964|
|      1|  33065|     4|  4.436782|
|      1|   5461|     3| 4.2342515|
|      1|  29703|     5|  5.485955|
|      1|  42404|     5| 4.6234093|
|      1|  33716|     5|  4.103488|
|      1|  25214|     4| 3.9475946|
|      1|  25164|     4| 3.9230397|
|      1|  31001|     4| 4.8696833|
|      1|   3662|     4| 4.3757577|
|      2|  11692|     3|  4.006947|
|      1|  51838|     5|  4.869015|
|      1|  37284|     5| 4.6060414|
|      1|   1185|     4| 3.7848606|
|      1|   5379|     5|  4.917064|
+-------+-------+------+----------+
only showing top 20 rows



In [ ]:
#Filtering user with user id "5461" with book id on which it has given the reviews
user1 = test_data.filter(test_data['user_id']==5461).select(['book_id','user_id'])
#Displaying user1 data
user1.show()

+-------+-------+
|book_id|user_id|
+-------+-------+
|      1|   5461|
|      5|   5461|
|      7|   5461|
|     28|   5461|
|     66|   5461|
|     70|   5461|
|     72|   5461|
|    121|   5461|
|    172|   5461|
|    186|   5461|
|    222|   5461|
|    233|   5461|
|    255|   5461|
|    273|   5461|
|    444|   5461|
|    461|   5461|
|    478|   5461|
|    531|   5461|
|    621|   5461|
|    639|   5461|
+-------+-------+
only showing top 20 rows



In [ ]:
user1.count()

35

In [ ]:
#Traning and evaluating for user1 with our model trained with the help of training data
recommendations = model.transform(user1)
#Displaying the predictions of books for user1
recommendations.orderBy('prediction',ascending=False).show()

+-------+-------+----------+
|book_id|user_id|prediction|
+-------+-------+----------+
|     72|   5461|  5.196121|
|    863|   5461| 4.8300543|
|    444|   5461| 4.8180566|
|   1094|   5461|  4.726987|
|    233|   5461| 4.5926175|
|     70|   5461|  4.573484|
|    461|   5461|  4.518782|
|    876|   5461| 4.4915648|
|      7|   5461|  4.486371|
|   1402|   5461| 4.4428067|
|    669|   5461|   4.42416|
|    255|   5461| 4.4104967|
|     28|   5461|  4.375878|
|    621|   5461|  4.338524|
|   1202|   5461|  4.275343|
|      1|   5461| 4.2342515|
|   1493|   5461| 4.2266264|
|    857|   5461|  4.190601|
|      5|   5461| 4.1859503|
|    478|   5461|  4.183007|
+-------+-------+----------+
only showing top 20 rows



In [ ]:
recommendations.show()

+-------+-------+----------+
|book_id|user_id|prediction|
+-------+-------+----------+
|      1|   5461| 4.2342515|
|      5|   5461| 4.1859503|
|      7|   5461|  4.486371|
|     28|   5461|  4.375878|
|     66|   5461|   4.11926|
|     70|   5461|  4.573484|
|     72|   5461|  5.196121|
|    121|   5461| 4.1503253|
|    172|   5461|   4.06039|
|    186|   5461| 4.0417895|
|    222|   5461|  4.040632|
|    233|   5461| 4.5926175|
|    255|   5461| 4.4104967|
|    273|   5461|  3.893582|
|    444|   5461| 4.8180566|
|    461|   5461|  4.518782|
|    478|   5461|  4.183007|
|    531|   5461|   3.56918|
|    621|   5461|  4.338524|
|    639|   5461| 4.1384583|
+-------+-------+----------+
only showing top 20 rows

